In [11]:
import torch

In [33]:
tensor_path = "/home/max/Studium/Leipzig/Semster6/Math_and_ML/math-ml/experiments/test_qwen_baseline/output_2025-06-24_16-22.pt"

exp_results = torch.load(tensor_path)

In [34]:
exp_results.keys()

dict_keys(['prompt0', 'prompt1', 'prompt2'])

In [35]:
exp_results['prompt0'].keys()

dict_keys(['generated_tokens', 'decoded_tokens', 'top_p_tokens', 'top_p_logits', 'top_p_probs', 'prompt', 'question', 'ground_truth', 'latency'])

In [42]:
prompt = 'prompt1'
print(f"Answer: {' '.join(exp_results[prompt]['decoded_tokens'])}")
print()
print(exp_results[prompt]['ground_truth'])

Answer:  
 A : 3  < eos

It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3


In [ ]:
exp_results['prompt1']['top_p_probs']

tensor([0.4160, 0.2227, 0.1270, 0.0771, 0.0530, 0.0498, 0.0118, 0.0087, 0.0036,
        0.0032, 0.0030, 0.0030, 0.0014, 0.0012, 0.0010, 0.0010, 0.0007, 0.0006,
        0.0006, 0.0006, 0.0006], dtype=torch.bfloat16)

In [45]:
exp_results['prompt0']['top_p_tokens'][9]

tensor([53308,  2055,   576])

In [46]:
exp_results['prompt0']['generated_tokens'][9]

tensor(53308)

In [40]:
len(exp_results['prompt0']['decoded_tokens'])

10

In [30]:
raw_test = 'The answer is 72.0'
test = 72.0

In [28]:
reverse = test[::-1]
print(reverse)

cba


In [34]:
test = str(test)
raw_test_rvs = raw_test[::-1]
for char in test:
    for idx,char_b in enumerate(raw_test_rvs):
        if char == char_b:
            indice = len(raw_test_rvs) - idx -1
            print(indice)
            break

14
15
16
17
